# 理解BP算法

直觉告诉我们：  
同一层中权重较大的连接在反向传播误差时分担了更多的误差

![fanxiang](images/反向传播误差.gif)

对于复杂的非线性问题（包含激活函数），想要让误差最小/得到损失函数最小值，
**梯度下降法**是一种很好的方法
- 优点：
    - 方法本身简单，要求低
    - 方法具有弹性，能够容忍不完善数据
- 缺点：
    - 整体收敛速度不一定最快

对于一个损失函数为**平方误差函数**，激活函数为**Sigmoid函数**的单隐层神经网络。
 
其隐藏层与输出层权重的更新值为：$\frac{\partial E}{\partial W_{j,k}} = -(t_k-o_k) \cdot o_k \cdot (1-o_k) \cdot o_j$  
其输入层与隐藏层权重的更新值为：$\frac{\partial E}{\partial W_{i,j}} = - e_j \cdot o_j \cdot (1-o_j) \cdot o_i$

# 使用单隐层神经网络完成手写数字识别

In [12]:
import numpy
import scipy.special

class neuralNetwork:
    # 定义神经网络
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # w11 w21
        # w12 w22
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        self.lr = learningrate
        
        # sigmoid
        self.activation_function = lambda x: scipy.special.expit(x)
        

    def train(self, inputs_list, targets_list):
        # 将输入转换为二维
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        
        hidden_inputs = numpy.dot(self.wih, inputs)
        
        hidden_outputs = self.activation_function(hidden_inputs)
        
        
        final_inputs = numpy.dot(self.who, hidden_outputs)
        
        final_outputs = self.activation_function(final_inputs)
        
        # 输出层误差为 target - 输出
        output_errors = targets - final_outputs
        # 隐层误差为 隐层输出层权重*输出层误差
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # 更新两层权重，根据公式
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        

    
    def query(self, inputs_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        hidden_inputs = numpy.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        final_inputs = numpy.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [4]:
# 初始化节点数量和学习率
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

learning_rate = 0.1

n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

In [5]:
# 加载数据集和测试集
with open("mnist_train.csv", 'r')as f:
    training_data_list = f.readlines()
with open("mnist_test.csv", 'r') as f2:
    test_data_list = f2.readlines()

In [6]:
epochs = 5

for e in range(epochs):
    for record in training_data_list:
        all_values = record.split(',')
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        targets = numpy.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)

In [7]:
# 计算准确率，使用得分板
scorecard = []
for record in test_data_list:
    all_values = record.split(',')
    correct_label = int(all_values[0])
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    outputs = n.query(inputs)
    label = numpy.argmax(outputs)
    if (label == correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0)

In [11]:
scorecard_array = numpy.asarray(scorecard)
print (f"performance = {scorecard_array.sum() / scorecard_array.size}")

performance = 0.9766


# 自动求导
- 自动求导计算一个函数在指定值上的导数
- 与符号求导、数值求导不同

- 符号求导
![fuhao](images/符号求导.png)
- 数值求导
![shuzhi](images/数值求导.png)

# 计算图
- 将公式分解成最小操作（元操作、操作子）
- 将计算表示成一个无环图

![DAG](images/无环图.png)


- 链式法则: $\frac{\partial y}{\partial x}=\frac{\partial y}{\partial u_n} \frac{\partial u_n}{\partial u_{n-1}} \ldots \frac{\partial u_2}{\partial u_1} \frac{\partial u_1}{\partial x}$
-正向累积 $$\frac{\partial y}{\partial x}=\frac{\partial y}{\partial u_n}\left(\frac{\partial u_n}{\partial u_{n-1}}\left(\ldots\left(\frac{\partial u_2}{\partial u_1} \frac{\partial u_1}{\partial x}\right)\right)\right)$$
-反向累积、又称反向传递
$$
\frac{\partial y}{\partial x}=\left(\left(\left(\frac{\partial y}{\partial u_n} \frac{\partial u_n}{\partial u_{n-1}}\right) \ldots\right) \frac{\partial u_2}{\partial u_1}\right) \frac{\partial u_1}{\partial x}
$$

### 计算单个变量梯度的复杂度
- 时间复杂度
    - 正向反向均为$O(n)$
- 空间复杂度
    - 正向$O(1)$
    - 反向$O(n)$用于存储正向计算的全部结果，**需要计算多个变量的梯度时效果更优**🌟


# 使用pytorch自动求导

In [134]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import pandas as pd


class MNISTDataSet(Dataset):
    def __init__(self,path):
        with open(path,'r') as f:
            csv = pd.read_csv(path,header=None)
        targets = csv.values[:,0]
        self.features = csv.values[:,1:]/255.0*0.99 + 0.01

        self.labels = numpy.zeros((targets.shape[0],10)) + 0.01
        for i in range(len(targets)):
            self.labels[i,int(targets[i])] = 0.99
        
    def __len__(self):
        return len(self.labels)
    def __getitem__(self,idx):
        return self.labels[idx],self.features[idx]
        
train_set = MNISTDataSet('mnist_train.csv')
test_set = MNISTDataSet('mnist_test.csv')
train_loader = DataLoader(train_set,batch_size=64,shuffle=True)
test_loader = DataLoader(test_set,batch_size=1,shuffle=True)

In [135]:
class SingleHiddenNerualNetwork(nn.Module):
    def __init__(self,input_num,hidden_num,output_num):
        super().__init__()
        self.layer1 = nn.Linear(in_features=input_num,out_features=hidden_num,bias=True)
        self.layer2 = nn.Linear(in_features=hidden_num,out_features=output_num,bias=True)
        
        
    def forward(self,x):
        x = self.layer1(x)
        x = torch.sigmoid(x)
        x = self.layer2(x)
        x = torch.sigmoid(x)
        return x
    
    def predict(self,x):
        x = torch.sigmoid(self.layer2(torch.sigmoid(self.layer1(x))))
        y = torch.argmax(x)
        return y

In [140]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.xavier_normal_(m.weight)
        nn.init.constant_(m.bias, 0.0)


net = SingleHiddenNerualNetwork(784,200,10)
net = net.double()
net.apply(weights_init)

epochs = 200
lr = 0.1
loss = nn.MSELoss()
updater = torch.optim.SGD(net.parameters(),lr=lr)

In [141]:
net.train()

import time
start = time.time()
# 训练开始
for i in range(epochs):
    for y,x in iter(train_loader):
        y_hat = net(x)
        l = loss(y_hat,y)
        updater.zero_grad()
        l.sum().backward()
        updater.step()
    print(f'epoch{i} loss:{l}')

end = time.time()
print(f'cost time:{end-start}')

epoch0 loss:0.08034828241029349
epoch1 loss:0.07670363993730017
epoch2 loss:0.067474609922055
epoch3 loss:0.06000419883749577
epoch4 loss:0.04718294404907688
epoch5 loss:0.03745918295145245
epoch6 loss:0.0393590795468021
epoch7 loss:0.03784484748513417
epoch8 loss:0.041346607476078864
epoch9 loss:0.030116500240849952
epoch10 loss:0.028333975697377056
epoch11 loss:0.025343488888708687
epoch12 loss:0.022716151022046455
epoch13 loss:0.02659029279208825
epoch14 loss:0.035229157848461236
epoch15 loss:0.030771429809598388
epoch16 loss:0.022124340451695833
epoch17 loss:0.018623662176440965
epoch18 loss:0.02282241545428494
epoch19 loss:0.027965213975814186
epoch20 loss:0.03331219396558747
epoch21 loss:0.03331885266981581
epoch22 loss:0.03032288276695269
epoch23 loss:0.023403952431129773
epoch24 loss:0.019845140228108328
epoch25 loss:0.01556962601845606
epoch26 loss:0.021563081489569293
epoch27 loss:0.024676564378801686
epoch28 loss:0.023322325628249742
epoch29 loss:0.017192547031382586
epoch30

In [142]:
# 检查测试集准确率
net.eval()
metric = [0,0]
for test_labels,test_features in iter(test_loader):
    test_labels = torch.argmax(test_labels)
    label_pre = net.predict(test_features)
#     print(label_pre.shape,label_pre)
    if label_pre == test_labels:
        metric[0]+=1        
    metric[1]+=1
print(f'acc:{metric[0]/metric[1]}')

acc:0.9345


# 小结
- 局部最小点
- 梯度下降慢、资源占用高
- 激活函数灵活